# 0. Install and Import Dependencies

In [ ]:
!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

# 1. Load Model

In [2]:
interpreter = tf.lite.Interpreter(model_path='D:\deep_learning\movenet\movenet.tflite')
interpreter.allocate_tensors()

In [3]:
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector

cap = cv2.VideoCapture(0)
detector = FaceMeshDetector(maxFaces=1)

# 2. Make Detections

In [22]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    success, img = cap.read()
    frame, faces = detector.findFaceMesh(frame, draw=False)
    # print(success)
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    if faces:
        face = faces[0]
        pointLeft = face[145]
        pointRight = face[374]
        # Drawing
        # cv2.line(img, pointLeft, pointRight, (0, 200, 0), 3)
        # cv2.circle(img, pointLeft, 5, (255, 0, 255), cv2.FILLED)
        # cv2.circle(img, pointRight, 5, (255, 0, 255), cv2.FILLED)
        w, _ = detector.findDistance(pointLeft, pointRight)
        W = 6.3

        # # Finding the Focal Length
        # d = 50
        # f = (w*d)/W
        # print(f)

        # Finding distance
        f = 750
        # print(d)
        dist = shoulder_dist(keypoints_with_scores,0)
        # print(dist)
        dist = list(dist)
        dist = int(dist[0])
        
        # should, _ = detector.findDistance(pointLeft, pointRight)
        d = (W * dist) / w


        cvzone.putTextRect(frame, f'Shoulder width: {int(d)}cm',
                           (face[10][0] - 100, face[10][1] - 50),
                           scale=2)

    
    
    

    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.7)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [21]:
dist = list(dist)
print(int(dist[0]))


338


In [23]:
y, x, c = frame.shape
shaped = np.squeeze(np.multiply(keypoints_with_scores, [y,x,1]))
shaped
print(y,x)

480 640


In [16]:
dist = shoulder_dist(keypoints_with_scores,0.5)
print(dist)

None


In [13]:
def shoulder_dist(keypoints_with_scores,confidence_threshold):
    left = np.squeeze(np.multiply(keypoints_with_scores[0][0][5],[480,640,1]))
    right = np.squeeze(np.multiply(keypoints_with_scores[0][0][6],[480,640,1]))
    y_l, x_l, conf_left = left
    y_r, x_r, conf_right = right
    l = [int(y_l),int(x_l)]
    r = [int(y_r),int(x_r)]
    dist = detector.findDistance(l, r)

    if (conf_left > confidence_threshold) & (conf_right > confidence_threshold):
        return dist


In [28]:
print(pointLeft)

[299, 276]


In [10]:
left = np.squeeze(np.multiply(keypoints_with_scores[0][0][5],[480,640,1]))
right = np.squeeze(np.multiply(keypoints_with_scores[0][0][6],[480,640,1]))
y_l, x_l, conf_left = left
y_r, x_r, conf_right = right
l = [int(y_l),int(x_l)]
r = [int(y_r),int(x_r)]
print(l,r)
dist = detector.findDistance(l, r)
dist

[420, 405] [424, 222]


(183.04371062672436, (420, 405, 424, 222, 422, 313))

In [26]:
kp = np.squeeze(np.multiply(keypoints_with_scores[0][0][5],[480,640,1]))
print(keypoints_with_scores[0][0][6])
ky, kx, kp_conf = kp
print((int(ky),int(kx)))
dist = detector.findDistance(pointLeft, )


[0.7652805  0.22074513 0.6345438 ]
(357, 537)


# 3. Draw Keypoints

In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    count = 0
    for kp in shaped:

        ky, kx, kp_conf = kp
        if count == 5 or count == 6:
            if kp_conf > confidence_threshold:
                cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1) 
        count+=1

# 4. Draw Edges

In [6]:
EDGES = {
  
    (5, 6): 'y'

}

In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)